# Test Individual Crews

This notebook allows you to test and debug each crew independently.

**Setup:**
1. Make sure your `.env` file has OPENAI_API_KEY
2. Run cells in order (or jump to specific crew)


In [ ]:
# Setup and imports
import json
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

# Add src to path
sys.path.insert(0, str(Path.cwd() / "src"))

# Load environment
load_dotenv()

# Set OpenAI model for CrewAI
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"  # or gpt-4o, gpt-4-turbo
# Verify API key is loaded
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY not found in environment!")
    print("   Please add it to your .env file")
else:
    print("✓ OpenAI API key loaded")
    print(f"✓ Using model: {os.getenv('OPENAI_MODEL_NAME')}")

print("✓ Setup complete")

In [ ]:
# Weaviate Configuration and Health Check
import weaviate

# Set Weaviate connection parameters
os.environ.setdefault("WEAVIATE_HOST", "localhost")
os.environ.setdefault("WEAVIATE_PORT", "8080")
os.environ.setdefault("WEAVIATE_GRPC_PORT", "50051")

print("Checking Weaviate connection...")
try:
    client = weaviate.connect_to_local(
        host=os.getenv("WEAVIATE_HOST"),
        port=int(os.getenv("WEAVIATE_PORT")),
        grpc_port=int(os.getenv("WEAVIATE_GRPC_PORT")),
    )

    # Check if connected
    if client.is_ready():
        print("✓ Weaviate is connected and ready")

        # Check if SecurityControl collection exists
        if client.collections.exists("SecurityControl"):
            collection = client.collections.get("SecurityControl")
            # Try to count objects (may not work in all versions)
            print("✓ SecurityControl collection exists")
        else:
            print("⚠️  WARNING: SecurityControl collection does not exist!")
            print("   Run: python -m security_requirements_system.tools.weaviate_setup")
    else:
        print("⚠️  WARNING: Weaviate is not ready")

    client.close()

except Exception as e:
    print(f"❌ ERROR: Cannot connect to Weaviate: {e}")
    print(f"   Make sure Weaviate is running on {os.getenv('WEAVIATE_HOST')}:{os.getenv('WEAVIATE_PORT')}")
    print("   Start with: docker-compose up -d")
    print("\nWeaviate Configuration:")
    print(f"  Host: {os.getenv('WEAVIATE_HOST')}")
    print(f"  Port: {os.getenv('WEAVIATE_PORT')}")
    print(f"  gRPC Port: {os.getenv('WEAVIATE_GRPC_PORT')}")

In [ ]:
# Helper functions


def display_crew_output(result, crew_name):
    """Display crew output in a readable format."""
    print(f"\n{'='*60}")
    print(f"{crew_name} OUTPUT")
    print(f"{'='*60}\n")

    if hasattr(result, "raw"):
        print("RAW OUTPUT:")
        print(result.raw[:500] + "..." if len(result.raw) > 500 else result.raw)

    if hasattr(result, "pydantic") and result.pydantic:
        print("\nSTRUCTURED OUTPUT:")
        print(json.dumps(result.pydantic.model_dump(), indent=2, default=str)[:1000])

    if hasattr(result, "tasks_output"):
        print(f"\nTASKS: {len(result.tasks_output)}")
        for i, task in enumerate(result.tasks_output):
            task_name = task.name if hasattr(task, "name") else f"Task {i}"
            print(f"  {i+1}. {task_name}")
            if hasattr(task, "pydantic") and task.pydantic:
                print(f"     Type: {type(task.pydantic).__name__}")


def save_output(result, crew_name, output_dir="test_outputs"):
    """Save crew output to JSON file."""
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)

    output_file = output_path / f"{crew_name}_output.json"

    data = {}
    if hasattr(result, "raw"):
        data["raw"] = result.raw
    if hasattr(result, "pydantic") and result.pydantic:
        data["pydantic"] = result.pydantic.model_dump()
    if hasattr(result, "tasks_output"):
        data["tasks"] = []
        for task in result.tasks_output:
            task_data = {"name": task.name if hasattr(task, "name") else "unknown", "raw": task.raw if hasattr(task, "raw") else str(task)}
            if hasattr(task, "pydantic") and task.pydantic:
                task_data["pydantic"] = task.pydantic.model_dump()
            data["tasks"].append(task_data)

    with open(output_file, "w") as f:
        json.dump(data, f, indent=2, default=str)

    print(f"\n💾 Saved to: {output_file}")
    return output_file


def load_cached_output(crew_name, output_dir="test_outputs"):
    """Load cached crew output from JSON file if it exists."""
    output_file = Path(output_dir) / f"{crew_name}_output.json"

    if output_file.exists():
        with open(output_file, "r") as f:
            data = json.load(f)
        print(f"📂 Loaded cached output from: {output_file}")
        return data
    else:
        print(f"⚠️  No cached output found for {crew_name}")
        return None


print("✓ Helper functions loaded")

## Loading Cached Outputs

Each crew cell below will automatically load required data from cached JSON files if the variables don't exist in memory. This allows you to:
- Jump to any crew without running previous cells
- Reuse outputs from previous runs
- Speed up testing

If a required cache file is missing, you'll see an error prompting you to run the prerequisite crew first.


In [ ]:
# Sample requirements text

SAMPLE_REQUIREMENTS = """
E-commerce Platform Requirements:

1. User Management
   - User registration and login
   - Multi-factor authentication
   - Password reset functionality
   - Profile management

2. Product Catalog
   - Browse products by category
   - Search functionality
   - Product details and images
   - Inventory management

3. Shopping Cart & Checkout
   - Add/remove items from cart
   - Secure payment processing (credit cards, PayPal)
   - Order confirmation emails
   - Shipping address management

4. Order Management
   - Order history
   - Order tracking
   - Returns and refunds

5. Admin Panel
   - User management
   - Product management
   - Order processing
   - Sales analytics
"""

print("✓ Sample requirements loaded")
print(f"Length: {len(SAMPLE_REQUIREMENTS)} characters")

In [ ]:
# Helper: Load all cached outputs (run this to load dependencies)


def load_all_dependencies():
    """Load all crew outputs from cache into memory."""
    global analysis_output, architecture_output, stakeholder_output
    global threat_output, domain_output, ai_security_output, compliance_output
    global security_architecture_output, roadmap_output, verification_output, validation_output
    global components_json, threats_json, security_controls_json

    from src.security_requirements_system.data_models import (
        AnalysisOutput,
        ArchitectureOutput,
        ThreatModelingOutput,
        DomainSecurityOutput,
    )

    # 1. Requirements Analysis
    if cached := load_cached_output("requirements_analysis"):
        analysis_task = next((t for t in cached["tasks"] if t["name"] == "analyze_requirements"), None)
        arch_task = next((t for t in cached["tasks"] if t["name"] == "analyze_architecture"), None)

        if analysis_task and "pydantic" in analysis_task:
            analysis_output = AnalysisOutput(**analysis_task["pydantic"])
        if arch_task and "pydantic" in arch_task:
            architecture_output = ArchitectureOutput(**arch_task["pydantic"])
            components_json = json.dumps([c for c in arch_task["pydantic"]["components"]])

    # 2. Stakeholder
    if cached := load_cached_output("stakeholder"):
        stakeholder_output = cached.get("raw", "")

    # 3. Threat Modeling
    if cached := load_cached_output("threat_modeling"):
        if "pydantic" in cached:
            threat_output = ThreatModelingOutput(**cached["pydantic"])
            threats_json = json.dumps(cached["pydantic"])

    # 4. Domain Security
    if cached := load_cached_output("domain_security"):
        if "tasks" in cached and len(cached["tasks"]) > 0:
            if "pydantic" in cached["tasks"][0]:
                domain_output = DomainSecurityOutput(**cached["tasks"][0]["pydantic"])
                security_controls_json = json.dumps(cached["tasks"][0]["pydantic"])

    # 5. AI/LLM Security
    if cached := load_cached_output("llm_security"):
        ai_security_output = cached.get("raw", "")

    # 6. Compliance
    if cached := load_cached_output("compliance"):
        compliance_output = cached.get("raw", "")

    # 7. Security Architecture
    if cached := load_cached_output("security_architecture"):
        security_architecture_output = cached.get("raw", "")

    # 8. Roadmap
    if cached := load_cached_output("roadmap"):
        roadmap_output = cached.get("raw", "")

    # 9. Verification
    if cached := load_cached_output("verification"):
        verification_output = cached.get("raw", "")

    # 10. Validation
    if cached := load_cached_output("validation"):
        if "tasks" in cached and len(cached["tasks"]) > 0:
            if "pydantic" in cached["tasks"][0]:
                from security_requirements_system.data_models import ValidationOutput

                validation_output = ValidationOutput(**cached["tasks"][0]["pydantic"])

    print("\n✓ All available dependencies loaded from cache")


# Uncomment to auto-load all cached outputs:
load_all_dependencies()

---

## 1. Requirements Analysis Crew

Analyzes product requirements and extracts security-relevant information.


In [ ]:
from security_requirements_system.crews.requirements_analysis_crew import AnalysisOutputCrew

print("Testing Requirements Analysis Crew...\n")

crew = AnalysisOutputCrew().crew()
result = crew.kickoff(inputs={"requirements_text": SAMPLE_REQUIREMENTS})

display_crew_output(result, "Requirements Analysis")
save_output(result, "requirements_analysis")

# Store outputs for downstream crews
analysis_task = next(filter(lambda x: x.name == "analyze_requirements", result.tasks_output))
architecture_task = next(filter(lambda x: x.name == "analyze_architecture", result.tasks_output))

analysis_output = analysis_task.pydantic
architecture_output = architecture_task.pydantic

print(f"\n✓ Application Summary: {analysis_output.application_summary[:150]}...")
print(f"✓ High-level requirements: {len(analysis_output.high_level_requirements)}")
print(f"✓ Architecture: {architecture_output.architecture_summary[:150]}...")

## 2. Stakeholder Crew

Identifies stakeholders and trust boundaries.


In [ ]:
from security_requirements_system.crews.stakeholder_crew import StakeholderCrew

print("Testing Stakeholder Crew...\n")

# Load dependencies from cache if not in memory
if "architecture_output" not in globals():
    print("⚠️  Loading dependencies from cache...")
    from security_requirements_system.data_models import ArchitectureOutput

    cached = load_cached_output("requirements_analysis")
    if cached and "tasks" in cached:
        arch_task = next((t for t in cached["tasks"] if t["name"] == "analyze_architecture"), None)
        if arch_task and "pydantic" in arch_task:
            architecture_output = ArchitectureOutput(**arch_task["pydantic"])

crew = StakeholderCrew().crew()
result = crew.kickoff(inputs={"requirements_text": SAMPLE_REQUIREMENTS, "architecture_summary": architecture_output.architecture_summary})

display_crew_output(result, "Stakeholder Analysis")
save_output(result, "stakeholder")

stakeholder_output = result.raw
print(f"\n✓ Output length: {len(stakeholder_output)} characters")

## 3. Threat Modeling Crew

Performs STRIDE threat modeling.


In [ ]:
from security_requirements_system.crews.threat_modeling_crew import ThreatModelingCrew

print("Testing Threat Modeling Crew...\n")

# Load dependencies from cache if not in memory
if "architecture_output" not in globals():
    print("⚠️  Loading dependencies from cache...")
    from security_requirements_system.data_models import ArchitectureOutput

    cached = load_cached_output("requirements_analysis")
    if cached and "tasks" in cached:
        arch_task = next((t for t in cached["tasks"] if t["name"] == "analyze_architecture"), None)
        if arch_task and "pydantic" in arch_task:
            architecture_output = ArchitectureOutput(**arch_task["pydantic"])

# Prepare components JSON
components_json = json.dumps([c.model_dump() for c in architecture_output.components]) if architecture_output.components else "[]"

crew = ThreatModelingCrew().crew()
result = crew.kickoff(
    inputs={
        "requirements_text": SAMPLE_REQUIREMENTS,
        "architecture_summary": architecture_output.architecture_summary,
        "components": components_json,
    }
)

display_crew_output(result, "Threat Modeling")
save_output(result, "threat_modeling")

threat_output = result.pydantic
threats_json = json.dumps(threat_output.model_dump())
print(f"\n✓ Threats identified: {len(threat_output.threats)}")
print(f"\nTop 5 threats:")
for i, threat in enumerate(threat_output.threats[:5], 1):
    print(f"{i}. {threat.threat_id}: {threat.description[:80]}...")

## 4. Domain Security Crew (OWASP ASVS)

Maps requirements to OWASP ASVS controls.


In [ ]:
from security_requirements_system.crews.domain_security_crew import DomainSecurityCrew

print("Testing Domain Security Crew...\n")

# Load dependencies from cache if not in memory
if "analysis_output" not in globals():
    print("⚠️  Loading dependencies from cache...")
    from security_requirements_system.data_models import AnalysisOutput

    cached = load_cached_output("requirements_analysis")
    if cached and "tasks" in cached:
        analysis_task = next((t for t in cached["tasks"] if t["name"] == "analyze_requirements"), None)
        if analysis_task and "pydantic" in analysis_task:
            analysis_output = AnalysisOutput(**analysis_task["pydantic"])

crew = DomainSecurityCrew().crew()
result = crew.kickoff(inputs={"high_level_requirements": json.dumps(analysis_output.high_level_requirements)})

display_crew_output(result, "Domain Security (OWASP)")
save_output(result, "domain_security")

domain_output = result.tasks_output[0].pydantic
security_controls_json = json.dumps(domain_output.model_dump())
total_controls = sum(len(rm.owasp_controls) for rm in domain_output.requirements_mapping)

print(f"\n✓ Requirements mapped: {len(domain_output.requirements_mapping)}")
print(f"✓ Total OWASP controls: {total_controls}")
print(f"✓ Recommended ASVS level: {domain_output.recommended_asvs_level}")

## 5. LLM Security Crew

Identifies AI/ML security requirements.


In [ ]:
from security_requirements_system.crews.llm_security_crew import LLMSecurityCrew

print("Testing LLM Security Crew...\n")

# Load dependencies from cache if not in memory
if "analysis_output" not in globals():
    print("⚠️  Loading dependencies from cache...")
    from security_requirements_system.data_models import AnalysisOutput

    cached = load_cached_output("requirements_analysis")
    if cached and "tasks" in cached:
        analysis_task = next((t for t in cached["tasks"] if t["name"] == "analyze_requirements"), None)
        if analysis_task and "pydantic" in analysis_task:
            analysis_output = AnalysisOutput(**analysis_task["pydantic"])

crew = LLMSecurityCrew().crew()
result = crew.kickoff(
    inputs={
        "requirements_text": SAMPLE_REQUIREMENTS,
        "analyzed_requirements": f"Application Summary: {analysis_output.application_summary}\nHigh-Level Requirements: {analysis_output.high_level_requirements}",
    }
)

display_crew_output(result, "LLM Security")
save_output(result, "llm_security")

ai_security_output = result.raw
print(f"\n✓ Output length: {len(ai_security_output)} characters")

## 6. Compliance Crew

Identifies compliance requirements (GDPR, PCI-DSS, etc.).


In [ ]:
from security_requirements_system.crews.compliance_crew import ComplianceCrew

print("Testing Compliance Crew...\n")

# Load dependencies from cache if not in memory
if "analysis_output" not in globals():
    print("⚠️  Loading dependencies from cache...")
    from security_requirements_system.data_models import AnalysisOutput

    cached = load_cached_output("requirements_analysis")
    if cached and "tasks" in cached:
        analysis_task = next((t for t in cached["tasks"] if t["name"] == "analyze_requirements"), None)
        if analysis_task and "pydantic" in analysis_task:
            analysis_output = AnalysisOutput(**analysis_task["pydantic"])

crew = ComplianceCrew().crew()
result = crew.kickoff(
    inputs={
        "requirements_text": SAMPLE_REQUIREMENTS,
        "analyzed_requirements": f"Application Summary: {analysis_output.application_summary}\nHigh-Level Requirements: {analysis_output.high_level_requirements}",
    }
)

display_crew_output(result, "Compliance")
save_output(result, "compliance")

compliance_output = result.raw
print(f"\n✓ Output length: {len(compliance_output)} characters")

## 7. Security Architecture Crew

Designs security architecture recommendations.


In [ ]:
from security_requirements_system.crews.security_architecture_crew import SecurityArchitectureCrew

print("Testing Security Architecture Crew...\n")

# Load dependencies from cache if not in memory
if "architecture_output" not in globals() or "components_json" not in globals() or "security_controls_json" not in globals():
    print("⚠️  Loading dependencies from cache...")
    from security_requirements_system.data_models import ArchitectureOutput, DomainSecurityOutput

    # Load architecture
    if "architecture_output" not in globals():
        cached = load_cached_output("requirements_analysis")
        if cached and "tasks" in cached:
            arch_task = next((t for t in cached["tasks"] if t["name"] == "analyze_architecture"), None)
            if arch_task and "pydantic" in arch_task:
                architecture_output = ArchitectureOutput(**arch_task["pydantic"])
                components_json = json.dumps([c.model_dump() for c in architecture_output.components])

    # Load security controls
    if "security_controls_json" not in globals():
        cached = load_cached_output("domain_security")
        if cached and "tasks" in cached and len(cached["tasks"]) > 0:
            if "pydantic" in cached["tasks"][0]:
                security_controls_json = json.dumps(cached["tasks"][0]["pydantic"])

crew = SecurityArchitectureCrew().crew()
result = crew.kickoff(
    inputs={
        "requirements_text": SAMPLE_REQUIREMENTS,
        "architecture_summary": architecture_output.architecture_summary,
        "components": components_json,
        "security_controls": security_controls_json,
    }
)

display_crew_output(result, "Security Architecture")
save_output(result, "security_architecture")

security_architecture_output = result.raw
print(f"\n✓ Output length: {len(security_architecture_output)} characters")

## 8. Implementation Roadmap Crew

Creates phased implementation plan.


In [ ]:
from security_requirements_system.crews.roadmap_crew import RoadmapCrew

print("Testing Roadmap Crew...\n")

# Load dependencies from cache if not in memory
if "security_controls_json" not in globals() or "threats_json" not in globals() or "compliance_output" not in globals():
    print("⚠️  Loading dependencies from cache...")

    # Load security controls
    if "security_controls_json" not in globals():
        cached = load_cached_output("domain_security")
        if cached and "tasks" in cached and len(cached["tasks"]) > 0:
            if "pydantic" in cached["tasks"][0]:
                security_controls_json = json.dumps(cached["tasks"][0]["pydantic"])

    # Load threats
    if "threats_json" not in globals():
        cached = load_cached_output("threat_modeling")
        if cached and "pydantic" in cached:
            threats_json = json.dumps(cached["pydantic"])

    # Load compliance
    if "compliance_output" not in globals():
        cached = load_cached_output("compliance")
        if cached:
            compliance_output = cached.get("raw", "")

crew = RoadmapCrew().crew()
result = crew.kickoff(
    inputs={
        "requirements_text": SAMPLE_REQUIREMENTS,
        "security_controls": security_controls_json,
        "threats": threats_json,
        "compliance_requirements": compliance_output,
    }
)

display_crew_output(result, "Implementation Roadmap")
save_output(result, "roadmap")

roadmap_output = result.raw
print(f"\n✓ Output length: {len(roadmap_output)} characters")

## 9. Verification Crew

Defines verification and testing strategy.


In [ ]:
from security_requirements_system.crews.verification_crew import VerificationCrew

print("Testing Verification Crew...\n")

# Load dependencies from cache if not in memory
if "security_controls_json" not in globals() or "compliance_output" not in globals():
    print("⚠️  Loading dependencies from cache...")

    # Load security controls
    if "security_controls_json" not in globals():
        cached = load_cached_output("domain_security")
        if cached and "tasks" in cached and len(cached["tasks"]) > 0:
            if "pydantic" in cached["tasks"][0]:
                security_controls_json = json.dumps(cached["tasks"][0]["pydantic"])

    # Load compliance
    if "compliance_output" not in globals():
        cached = load_cached_output("compliance")
        if cached:
            compliance_output = cached.get("raw", "")

crew = VerificationCrew().crew()
result = crew.kickoff(
    inputs={
        "security_controls": security_controls_json,
        "compliance_requirements": compliance_output,
        "owasp_controls": security_controls_json,
    }
)

display_crew_output(result, "Verification & Testing")
save_output(result, "verification")

verification_output = result.raw
print(f"\n✓ Output length: {len(verification_output)} characters")

## 10. Validation Crew

Validates completeness and quality of generated requirements.


In [ ]:
from security_requirements_system.crews.validation_crew import ValidationCrew

print("Testing Validation Crew...\n")

# Load dependencies from cache if not in memory
if "analysis_output" not in globals() or "domain_output" not in globals():
    print("⚠️  Loading dependencies from cache...")
    from security_requirements_system.data_models import AnalysisOutput, DomainSecurityOutput

    # Load requirements analysis
    if "analysis_output" not in globals():
        cached = load_cached_output("requirements_analysis")
        if cached and "tasks" in cached:
            analysis_task = next((t for t in cached["tasks"] if t["name"] == "analyze_requirements"), None)
            if analysis_task and "pydantic" in analysis_task:
                analysis_output = AnalysisOutput(**analysis_task["pydantic"])

    # Load domain security
    if "domain_output" not in globals():
        cached = load_cached_output("domain_security")
        if cached and "tasks" in cached and len(cached["tasks"]) > 0:
            if "pydantic" in cached["tasks"][0]:
                domain_output = DomainSecurityOutput(**cached["tasks"][0]["pydantic"])

crew = ValidationCrew().crew()
result = crew.kickoff(
    inputs={
        "requirements_text": SAMPLE_REQUIREMENTS,
        "analyzed_requirements": f"Application Summary: {analysis_output.application_summary}\\nHigh-Level Requirements: {analysis_output.high_level_requirements}",
        "security_controls": domain_output.model_dump_json(indent=2),
        "ai_security": "No AI components detected",
        "compliance_requirements": "PCI-DSS, GDPR",
    }
)

display_crew_output(result, "Validation")
save_output(result, "validation")

validation_output = result.tasks_output[0].pydantic
print(f"\n✓ Validation score: {validation_output.overall_score:.2f}")
print(f"✓ Validation passed: {validation_output.validation_passed}")
print(f"\nDimension scores:")
for dim, score in validation_output.dimension_scores.items():
    print(f"  - {dim}: {score:.2f}")

## Summary

All crew outputs saved to `test_outputs/` directory.

Run any cell above to test specific crews. Outputs are cached to JSON files for inspection.


In [ ]:
# List all saved outputs
import os

output_dir = Path("test_outputs")
if output_dir.exists():
    print("\nSaved outputs:")
    for file in sorted(output_dir.glob("*.json")):
        size = file.stat().st_size / 1024  # KB
        print(f"  - {file.name} ({size:.1f} KB)")
else:
    print("\nNo outputs saved yet. Run cells above to generate outputs.")